In [1]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import (
    VGG19, vgg19,
    VGG16, vgg16,
    ResNet50V2, resnet_v2,
    InceptionV3, inception_v3,
    DenseNet121, densenet,
    MobileNet, mobilenet,
    MobileNetV2, mobilenet_v2,
    Xception, xception,
    EfficientNetB0, efficientnet,
    EfficientNetB7, efficientnet,
    MobileNetV3Large, mobilenet_v3
)
from ModelTrainer import ModelTrainer
import random

In [2]:
model_details = [
    (VGG19, vgg19.preprocess_input, (224, 224)),
    # (MobileNetV3Large, mobilenet_v3.preprocess_input, (224, 224)),
    #(VGG16, vgg16.preprocess_input, (224, 224)),
    # (DenseNet121, densenet.preprocess_input, (224, 224)),
    # (InceptionV3, inception_v3.preprocess_input, (299, 299)),
    # (Xception, xception.preprocess_input, (299, 299)),
    # (ResNet50V2, resnet_v2.preprocess_input, (224, 224)),
    # (MobileNetV2, mobilenet_v2.preprocess_input, (224, 224)),
    # (MobileNet, mobilenet.preprocess_input, (224, 224)),
    # (EfficientNetB0, efficientnet.preprocess_input, (224, 224)),
    # (EfficientNetB7, efficientnet.preprocess_input, (600, 600)),
]

In [3]:
param_grid = {
    'optimizer': ['sgd', 'rmsprop', 'adam'],    
    'learning_rate_feature_extraction': [5e-5, 1e-4, 5e-4, 1e-3],
    'learning_rate_fine_tuning': [1e-4, 5e-4, 1e-3, 5e-3],
    'dropout_rate': [0.3, 0.5],  
    'dense_units': [256, 512, 1024], 
    'l2_reg': [0., 0.01],
    'batch_size': [16, 32],  
    'epochs': [100, 125, 150],

    'unfreeze_percentage': [0.2, 0.3, 0.4],

    # Early Stopping and Learning Rate Scheduler Tuning
    'patience_es_feature_extraction': [12, 14, 16],
    'patience_es_fine_tuning': [12, 14, 16],  # Early stopping patience
    'patience_lr': [5, 7, 10],  # Learning rate reduce patience (always < patience_es)
    'min_lr': [1e-6, 1e-5], 
    'factor_lr': [0.2, 0.3, 0.5, 0.7],  # Factor by which learning rate is reduced   

    # SGD specific parameters
    'momentum': [0.9],
    'nesterov': [True], 

    # Adam specific parameters
    'beta_1': [0.9],  
    'beta_2': [0.999],  
    'epsilon': [1e-8], 
    'amsgrad': [False, True],  

    # RMSprop specific parameters
    'rho': [0.9, 0.95],  
    'centered': [False, True],  
}


n_random_search = 5

In [4]:
train_dir = 'data_splitted_v2/train'
val_dir = 'data_splitted_v2/val'
save_dir = 'saved_models_v4/'

In [5]:
# best performance
best_val_loss = float('inf')
best_hyperparams = None
best_val_accuracy_at_min_loss = None 

model_hyperparam_map = []

In [6]:
with tf.device('/GPU:0'):
    # Random search
    for i in range(n_random_search):
    
        hyperparameters = {key: (random.choice(values) if len(values) > 1 else values[0]) for key, values in param_grid.items()}
        
        print(f"Iteration {i+1}/{n_random_search} with hyperparameters: {hyperparameters}")

        trainer = ModelTrainer(train_dir=train_dir, val_dir=val_dir, save_dir=save_dir, hyperparams=hyperparameters)

        for base_model_func, preprocess_input, image_size in model_details:
            train_generator, val_generator = trainer.get_generators(preprocess_input, image_size)
            num_classes = train_generator.num_classes

            base_model, model = trainer.build_model(base_model_func, image_size, num_classes)
            history = trainer.train_model(model, base_model, train_generator, val_generator, i+1)

            
            model_name = base_model_func.__name__
        
            val_losses = history['val_loss']
            val_accuracies = history['val_accuracy']
            
            min_val_loss = min(val_losses)
            min_val_loss_index = val_losses.index(min_val_loss)
            corresponding_val_accuracy = val_accuracies[min_val_loss_index]
            
            model_hyperparam_map.append({
                'model': model_name,
                'hyperparameters': hyperparameters,
                'val_loss': min_val_loss,
                'val_accuracy_at_min_loss': corresponding_val_accuracy
            })
            
            if min_val_loss < best_val_loss:
                best_val_loss = min_val_loss
                best_hyperparams = hyperparameters
                best_val_accuracy_at_min_loss = corresponding_val_accuracy
                
                trainer.save_history(history, model)


Iteration 1/5 with hyperparameters: {'optimizer': 'adam', 'learning_rate_feature_extraction': 0.0001, 'learning_rate_fine_tuning': 0.0001, 'dropout_rate': 0.3, 'dense_units': 256, 'l2_reg': 0.0, 'batch_size': 16, 'epochs': 125, 'unfreeze_percentage': 0.4, 'patience_es_feature_extraction': 12, 'patience_es_fine_tuning': 16, 'patience_lr': 5, 'min_lr': 1e-06, 'factor_lr': 0.3, 'momentum': 0.9, 'nesterov': True, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-08, 'amsgrad': False, 'rho': 0.9, 'centered': False}
Found 3749 images belonging to 3 classes.
Found 1225 images belonging to 3 classes.
Starting training for VGG19
Epoch 1/50
235/235 [==============================] - ETA: 0s - loss: 1.0433 - accuracy: 0.4473
Epoch 1: val_loss improved from inf to 0.94028, saving model to saved_models_v4/VGG19\iteration-01-epoch-01-val_loss-0.940-val_acc-0.562.h5
235/235 [==============================] - 55s 218ms/step - loss: 1.0433 - accuracy: 0.4473 - val_loss: 0.9403 - val_accuracy: 0.5624 - lr: 

In [ ]:
for entry in model_hyperparam_map:
    print(f"Model: {entry['model']}, Hyperparameters: {entry['hyperparameters']}, "
          f"Best Validation Loss: {entry['val_loss']}, Validation Accuracy at Min Loss: {entry['val_accuracy_at_min_loss']}")

In [ ]:
print(f"\nBest overall validation loss: {best_val_loss}")
print(f"Validation accuracy at best validation loss: {best_val_accuracy_at_min_loss}")
print(f"Best hyperparameters: {best_hyperparams}")

In [ ]:
# Model training
trainer = ModelTrainer(train_dir=train_dir, val_dir=val_dir, save_dir=save_dir, hyperparams=hyperparameters)
for base_model_func, preprocess_input, image_size in model_details:
    train_generator, val_generator = trainer.get_generators(preprocess_input, image_size)
    num_classes = train_generator.num_classes
    model = trainer.build_model(base_model_func, image_size, num_classes)
    history = trainer.train_model(model, train_generator, val_generator)
    trainer.save_history(history, model)